# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [2]:
# EW vulnerable.
vuln_ns, vuln_ew = False, True
# you sit North and hold:
hand = 'QT5.T8.KQT87.864'
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 3, False)

# the auction goes:
auction = ["PAD_START","PAD_START","PAD_START","PASS"]
bid = bot_bid.bid(auction)

auction = ["PAD_START","PAD_START","PAD_START","PASS", "PASS", "1N", "2C", "PASS"]
# what's your bid?
bid = bot_bid.bid(auction)


In [3]:
bid.samples

['QTx.T8.KQT8x.8xx K9.AQx.Axxx.QTxx AJ8xx.Kxxxx.9.AK xxx.J9x.Jxx.J9xx 0.78538 \n P-P-1N-2C-P-P-P (6.7)  \n P-P-1N-2C-P-2H-P-2S-P-P-P (9.9)  \n P-P-1N-2C-P-2S-P-P-P (10.0) ',
 'QTx.T8.KQT8x.8xx Axx.Kxx.AJx.KQTx KJ98x.AQ9xx.x.Ax xx.Jxx.9xxx.J9xx 0.78068 \n P-P-1N-2C-P-P-P (6.0)  \n P-P-1N-2C-P-2H-P-2S-P-P-P (9.7)  \n P-P-1N-2C-P-2S-P-P-P (9.8) ',
 'QTx.T8.KQT8x.8xx AK9.A9xx.AJxx.9x J8xx.KJx.9.AKJTx xxx.Qxxx.xxx.Qxx 0.75251 \n P-P-1N-2C-P-P-P (7.8)  \n P-P-1N-2C-P-2H-P-P-P (5.1)  \n P-P-1N-2C-P-2S-P-P-P (7.3) ',
 'QTx.T8.KQT8x.8xx 8xx.AQxx.A9x.KQT A9xx.KJx.x.AJ9xx KJx.9xxx.Jxxx.xx 0.74330 \n P-P-1N-2C-P-P-P (8.3)  \n P-P-1N-2C-P-2H-P-P-P (5.8)  \n P-P-1N-2C-P-2S-P-P-P (8.2) ',
 'QTx.T8.KQT8x.8xx AK.J9x.AJxx.Kxxx J98xx.KQxxx..AJ9 xxx.Axx.9xxx.QTx 0.74150 \n P-P-1N-2C-P-P-P (5.5)  \n P-P-1N-2C-P-2H-P-P-P (7.0)  \n P-P-1N-2C-P-2S-P-P-P (7.8) ',
 'QTx.T8.KQT8x.8xx xx.AKJ.Axxx.Axxx AJ9xx.Q9xxx.x.KQ K8x.xxx.J9x.JT9x 0.72056 \n P-P-1N-2C-P-P-P (4.4)  \n P-P-1N-2C-P-2H-P-P-P (7.0)  \n P-P-1N-2C-P

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START","PAD_START","PAD_START","PASS", "PASS", "1N", "2C", "PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

QTx.T8.KQT8x.8xx K9.AQx.Axxx.QTxx AJ8xx.Kxxxx.9.AK xxx.J9x.Jxx.J9xx
QTx.T8.KQT8x.8xx Axx.Kxx.AJx.KQTx KJ98x.AQ9xx.x.Ax xx.Jxx.9xxx.J9xx
QTx.T8.KQT8x.8xx AK9.A9xx.AJxx.9x J8xx.KJx.9.AKJTx xxx.Qxxx.xxx.Qxx
QTx.T8.KQT8x.8xx 8xx.AQxx.A9x.KQT A9xx.KJx.x.AJ9xx KJx.9xxx.Jxxx.xx
QTx.T8.KQT8x.8xx AK.J9x.AJxx.Kxxx J98xx.KQxxx..AJ9 xxx.Axx.9xxx.QTx
QTx.T8.KQT8x.8xx xx.AKJ.Axxx.Axxx AJ9xx.Q9xxx.x.KQ K8x.xxx.J9x.JT9x
QTx.T8.KQT8x.8xx AK9.K9xx.Axx.J9x J8x.AQxx.9.AKTxx xxxx.Jxx.Jxxx.Qx
QTx.T8.KQT8x.8xx Kxxx.AKxx.J9x.Ax Ax.9xxx.Ax.KQJT9 J98x.QJx.xxx.xxx
QTx.T8.KQT8x.8xx Kxx.K9x.AJx.AQTx AJ8xx.AQJxxx.x.J 9x.xx.9xxx.K9xxx
QTx.T8.KQT8x.8xx AJx.Q9x.9x.AKJxx K9xxx.Kxxxx.Ax.Q 8x.AJx.Jxxx.T9xx
QTx.T8.KQT8x.8xx xxx.AKxx.Axxx.Ax AK9x.Q9xx..KJTxx J8x.Jxx.J9xx.Q9x
QTx.T8.KQT8x.8xx KJ9x.AK9x.AJ9.Tx A8x.Jxxx.x.KQJxx xxx.Qxx.xxxx.A9x
QTx.T8.KQT8x.8xx A8x.9xxx.A9x.AKQ KJxxx.KQJx.J.T9x 9x.Axx.xxxx.Jxxx
QTx.T8.KQT8x.8xx K9x.KQx.xxx.AKTx AJ8xxx.AJ9x.A9x. x.xxxx.Jx.QJ9xxx
QTx.T8.KQT8x.8xx KJx.A9xx.A9x.A9x Axx.KQxx.x.KQJ